**!!!!!!!!!!!!! RUN AND RESTART SESSION !!!!!!!!!!!!!!!**

In [2]:
## !!!!!!!!!!!!! RUN AND RESTART SESSION !!!!!!!!!!!!!!!

%%capture
!pip install bitsandbytes accelerate transformers-interpret shap inseq
!pip install -U captum
!pip install transformers
!pip install --upgrade scipy==1.12
!pip install -U datasets

## !!!!!!!!!!!!! RUN AND RESTART SESSION !!!!!!!!!!!!!!!

In [14]:
import inseq
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

### Load Dataset and sample

In [7]:
dataset = load_dataset("EdinburghNLP/xsum")

In [19]:
# Few-shot examples from training set
train_examples = dataset["train"].select(range(2))

# Sample a subset of the test set for evaluation
test_sample = dataset["test"].select(range(1))
references = [example["summary"] for example in test_sample]

### Load MODEL and Tokenizer

In [30]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=2000)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### Zero-Shot, One-Shot and Few-Shot

In [16]:
def build_zero_shot_prompt(doc):
    """Builds a zero-shot prompt."""
    prompt_template = "Please write a short summary of the following article:\n\n{}"
    return prompt_template.format(doc)

def build_one_shot_prompt(doc, train_example):
    """Builds a one-shot prompt with one example."""
    prompt = ""
    prompt += "Task: Summarize the input text. An example is provided below. \n"
    prompt += f"### EXAMPLE:\nDocument: {train_example['document'].strip()}\nSummary: {train_example['summary'].strip()}\n\n"
    prompt += f"### INPUT TEXT:\nDocument: {doc.strip()}\nSummary:[Fill the summary]"
    return prompt

def build_few_shot_prompt(doc, few_shots):
    """Builds a few-shot prompt with multiple examples."""
    prompt = ""
    prompt += "Task: Summarize the input text. Examples are provided below. \n"
    for ex in few_shots:
        prompt += f"### EXAMPLE:\nDocument: {ex['document'].strip()}\nSummary: {ex['summary'].strip()}\n\n"
    prompt += f"### INPUT TEXT:\nDocument: {doc.strip()}\nSummary:[Fill the summary]"
    return prompt

In [27]:
document = test_sample[0]["document"]

zero_shot_prompt = build_zero_shot_prompt(document)
one_shot_prompt = build_one_shot_prompt(document, train_examples[0])
few_shot_prompt = build_few_shot_prompt(document, train_examples)

### Explanation of different promptings

In [60]:
qa_model = inseq.load_model("google/flan-t5-base", "input_x_gradient", tokenizer=tokenizer)
out = qa_model.attribute(few_shot_prompt, attribute_target=True, step_scores=["probability"], generation_args={"max_new_tokens": 128})

The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Attributing with input_x_gradient...: 100%|██████████| 32/32 [00:28<00:00,  1.09it/s]


In [61]:
# Print output
generated_output = out.info["generated_texts"][0]
print(generated_output)

A charity has said it is "disappointing" that some prison leavers are not getting the support they need to find accommodation.


In [62]:
# Aggregate only source tokens, leave target tokens as they are
out.aggregate("subwords", aggregate_target=False).show()

,▁A,▁charity,▁has,▁said,▁it,▁is,"▁""",d,is,a,p,pointing,"""",▁that,▁some,▁prison,▁leave,r,s,▁are,▁not,▁getting,▁the,▁support,▁they,▁need,▁to,▁find,▁accommodation,.,</s>
▁Task:,0.003,0.002,0.002,0.003,0.005,0.002,0.006,0.003,0.003,0.005,0.001,0.002,0.002,0.002,0.003,0.001,0.002,0.003,0.003,0.002,0.001,0.002,0.002,0.003,0.001,0.002,0.002,0.003,0.001,0.004,0.009
▁Summarize,0.005,0.003,0.003,0.004,0.002,0.002,0.004,0.003,0.002,0.002,0.002,0.001,0.004,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.003,0.002,0.002,0.002,0.002,0.002,0.002,0.004,0.019
▁the,0.001,0.001,0.001,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002
▁input,0.003,0.002,0.002,0.002,0.002,0.001,0.002,0.002,0.001,0.001,0.001,0.001,0.002,0.001,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.009
▁text.,0.004,0.002,0.002,0.003,0.002,0.002,0.003,0.002,0.001,0.002,0.001,0.001,0.003,0.002,0.002,0.001,0.002,0.001,0.001,0.001,0.001,0.002,0.002,0.001,0.002,0.001,0.002,0.001,0.001,0.003,0.008
▁Examples,0.004,0.003,0.002,0.002,0.001,0.002,0.002,0.002,0.001,0.002,0.002,0.001,0.002,0.001,0.002,0.001,0.001,0.001,0.001,0.001,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.013
▁are,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.0,0.0,0.0,0.0,0.001,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.003
▁provided,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.001,0.0,0.001,0.001,0.001,0.001,0.001,0.0,0.001,0.001,0.001,0.001,0.006
▁below.,0.002,0.002,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.008
▁###,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.015
